## Select features and K value

In [109]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import chain, combinations

df = pd.read_csv("data_by_genres.csv")

y = np.array(df['popularity'])
df = df.drop('genres', axis=1)
df = df.drop('popularity', axis=1)

### get all combinations of features (total: 4095)
index = []
def powerset(list_name):
    s = list(list_name)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

for x in powerset(df.columns):
    index.append(list(x))

X=[]
for i in range(1,len(index)): #index[0] is empty
    X.append(np.array(df[index[i]]))

P_x = np.arange(1, 200, 2).tolist()
V=[]
feature_combinations = []
accuracies = []
k_values = []

for i in range(1, len(X)):
    feature_combinations.append(index[i])
    accuracy_values = []
    k_values_subset = []
    
    #print(f"now i: {i}")
    
    for j in range(1, 200, 2):
        train_data, test_data, train_label, test_label = train_test_split(X[i], y, test_size=0.3)
        knn = KNeighborsRegressor(n_neighbors=j)
        knn.fit(train_data, train_label)
        accuracy = knn.score(test_data, test_label)
        accuracy_values.append(accuracy)
        k_values_subset.append(j)
  
    accuracies.append(accuracy_values)
    k_values.append(k_values_subset)

accuracies = np.array(accuracies)
max_accuracy_index = np.unravel_index(np.argmax(accuracies), accuracies.shape)
max_accuracy_features = feature_combinations[max_accuracy_index[0]]
max_accuracy = accuracies[max_accuracy_index[0]][max_accuracy_index[1]]
best_k_value = k_values[max_accuracy_index[0]][max_accuracy_index[1]]

print("最大準確率的特徵組合:", max_accuracy_features)
print("最大準確率:", max_accuracy)
print("最大準確率的K值:", best_k_value)


最大準確率的特徵組合: ['acousticness', 'danceability', 'instrumentalness', 'speechiness', 'tempo']
最大準確率: 0.6367836133843041
最大準確率的K值: 15


### Feature selection
- Selected features are: <font color=#bd3939>acousticness, danceability, instrumentalness, speechiness, tempo<font>
- Selected K value is: <font color=#bd3939>15<font>

### TEST

In [174]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

df = pd.read_csv('../data_by_genres.csv')

# 使用選定的特徵 ['acousticness', 'danceability', 'instrumentalness', 'speechiness', 'tempo']
features = ['acousticness', 'danceability', 'instrumentalness', 'speechiness', 'tempo']
X = df[features].values
y = df['genres'].values

### 特徵標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

knn = KNeighborsClassifier(n_neighbors=15)

knn.fit(X_scaled, y)

# 指定的音樂類型
target = ['alt z', 'electropop', 'pop']



# 預測最相關的3個genre
ret_genres = []

for i in range(len(target)):
    try:
        target_idx = list(df['genres']).index(target[i])
        target_features = X_scaled[target_idx].reshape(1, -1)
        nearest_neighbors = knn.kneighbors(target_features, n_neighbors=4, return_distance=False)

        # 最相關的音樂類型
        most_similar_genres = [y[i] for i in nearest_neighbors[0]]
        for tmp in most_similar_genres[1:]:
            ret_genres.append(tmp)

        #ret_genres.append(most_similar_genres[1:])
        #print(f'與{target[i]}最相近的三個音樂種類:', most_similar_genres[1:])
    
    except:
        print(f"'{target[i]}' doesn't exist in dataset.")
print(ret_genres)

'alt z' doesn't exist in dataset.
['indie poptimism', 'geek rock', 'dance pop', 'geek rock', 'latin talent show', 'british soul']
